-------------
# Chapter 6 - CAPM & APT
----------

In [43]:
using JuMP, HiGHS, Ipopt, Plots, LinearAlgebra, Printf, Statistics;

------
#### 1 
Let us assume a normal distribution of returns and risk-averse utility functions. Under what
conditions will all investors demand the same portfolio of risky assets?

--------
#### 2
The following data have been developed for the Donovan Company, the manufacturer of an
advanced line of adhesives:

| State | Probability | Market Return $R_m$ | Return for the Firm $R_j$ |
| --- | --- | --- | --- |
| 1 | .1 | −.15 | −.30 |
| 2 | .3 | .05 |.00 |
| 3 | .4 | .15 |.20 |
| 4 | .2 | .20 |.50 |


The risk-free rate is 6%. Calculate the following:

(a) The expected market return.

(b) The variance of the market return.

(c) The expected return for the Donovan Company.

(d) The covariance of the return for the Donovan Company with the market return.

(e) Write the equation of the security market line.

(f) What is the required return for the Donovan Company? How does this compare with its expected return?

In [92]:
P = [.1 .3 .4 .2]
Rm = [-.15 .05 .15 .2]
Rj = [-.3 0 .2 .5]
Rf = .06

ERm = P*Rm'
var_Rm = P*(Rm.-ERm)'.^2
ERj = P*Rj'
cov_mj = P*diag((Rm.-ERm)'*(Rj.-ERj))
Beta = cov_mj/var_Rm

@printf("Expectation of Market Return: %.2f \nExpectation of Firm Return: %.2f", ERm[1], ERj[1])
@printf("\nVariance of Market Return: %.2f \nVariance of Firm Return: %.4f \nCovariance of Market & Firm: %.4f", var_Rm[1], var_Rj[1], cov_mj[1])
@printf("\nBeta of Firm: %.2f", Beta[1])

Expectation of Market Return: 0.10 
Expectation of Firm Return: 0.15
Variance of Market Return: 0.01 
Variance of Firm Return: 0.0525 
Covariance of Market & Firm: 0.0215
Beta of Firm: 2.15

Equation of the SML: 
$$
\begin{align*}
E[R_j] &= R_f + \beta_i \left(E\left[R_m\right] - R_f\right) \\
&= 0.06 + \beta_i \left(0.1 - 0.06\right) \\
&= 0.06 + 0.04 \cdot \beta_i
\end{align*}
$$

$$\begin{align*}
    E[R_j] &= R_f + \beta_j \left(E\left[R_m\right] - R_f\right) \\
    &= 0.06 + 2.15\left(0.1 - 0.06\right) \\
    &= 0.06 + 2.15\left(0.1 - 0.06\right) \\
    &= 0.146
\end{align*}$$

This is below our Expected Return of 0.15. This implies the firm is undervalued & we should therefore purchase stocks of this security.

--------------
#### 3

The following data have been developed for the Milliken Company:

| Year | Market Return | Company Returns |
| --- | --- | --- |
| 1978 | .27 | .25 |
| 1977 | .12 | .05 |
| 1976 | −.03 | −.05
| 1975 | .12 | .15 |
| 1974 | −.03 | −.10
| 1973 | .27 | .30 |

The yield to maturity on Treasury bills is .066 and is expected to remain at this point for the
foreseeable future. Calculate the following:

(a) The expected market return.

(b) The variance of the market return.

(c) The expected rate of return for the Milliken Company.

(d) The covariance of the return for the Milliken Company with the return on the market.

(e) Write the equation of the security market line.

(f) What is the required return for the Milliken Company?

In [91]:
R_m = [.27 .12 -.03 .12 -.03 .27]
ER_m = 1/length(R_m)*R_m*ones(6)

var_m = repeat([1/length(R_m)], length(R_m))'*diag((R_m.-ER_m)'*(R_m.-ER_m))

R_j = [.25 .05 -.05 .15 -.1 .3]
ER_j = 1/length(R_j)*R_j*ones(6)

cov_mj = repeat([1/length(R_m)], length(R_m))'*diag((R_m.-ER_m)'*(R_j.-ER_j))

@printf("Expectation of Market Return: %.2f \nExpectation of Milliken Return: %.2f", ER_m[1], ER_j[1])
@printf("\nVariance of Market Return: %.2f \nCovariance of Market & Milliken: %.4f", var_m[1], cov_mj[1])

Expectation of Market Return: 0.12 
Expectation of Milliken Return: 0.10
Variance of Market Return: 0.01 
Covariance of Market & Milliken: 0.0175

Equation of the SML: 
$$
\begin{align*}
E[R_j] &= R_f + \beta_i \left(E\left[R_m\right] - R_f\right) \\
&= 0.066 + \beta_i \left(0.12 - 0.066\right) \\
&= 0.066 + 0.054 \cdot \beta_i
\end{align*}
$$

In [94]:
var_j = repeat([1/length(R_j)], length(R_j))'*diag((R_j.-ER_j)'*(R_j.-ER_j))
Beta = var_j/cov_mj

1×1 transpose(::Vector{Float64}) with eltype Float64:
 1.0077519379844964

Required Return on Milliken:
$$
\begin{align*}
E\left[R_j\right] &= 0.066 + 0.054 \cdot \beta_i \\
&= 0.066 + 0.054 \cdot 1.00775 \\
&= 0.1204
\end{align*}
$$

Implies Milliken currently overvalued, so we would expect price to fall.

----------
#### 4

For the data in Table Q4 perform the indicated calculations.

| | $P_t$ | $\frac{P_t}{P_{t-1}}-1$ | $\frac{Div_t}{P_t}$ | $R_{mt}$ | $R_{mt}-\bar{R}_m$ | $(R_{mt}-\bar{R}_m)^2$ | $R_f$ |
| --- | --- | --- | --- | --- | --- | --- |--- |
| 1960 | 55.84 |
| 1961 | 66.27 | 0.1868 | .0298 | 0.2166 | 0.1386 | 0.0192 |.03 |
| 1962 | 62.38 | -0.0587 | .0337 | -0.025 | -0.103 | 0.0106 |.03 |
| 1963 | 69.87 | 0.12 | .0317 | 0.1518 | 0.074 | 0.0054 |.03 |
| 1964 | 81.37 | 0.1646 | .0301 | 0.1947 | 0.1167 | 0.0136 |.04 |
| 1965 | 88.17 | 0.0836 | .0300 | 0.1136 | 0.0356 | 0.0013 |.04 |
| 1966 | 85.26 | -0.033 | .0340 | 0.001 | -0.077 | 0.0059 |.04 |
| 1967 | 91.93 | 0.0782 | .0320 | 0.1102 | 0.0323 | 0.001 |.05 |
| 1968 | 98.70 | 0.0736 | .0307 | 0.1043 | 0.0264 | 0.0007 |.05 |
| 1969 | 97.84 | -0.0087 | .0324 | 0.0237 | -0.0543 | 0.0029 |.07 |
| 1970 | 83.22 | -0.1494 | .0383 | -0.1111 | 0.1891 | 0.0358 |.06 |

a. $\bar{R}_m$ = 0.0457

b. $VAR(R_m)$ = 0.00965

c. $σ(R_m)$ = 0.0982

In [121]:
P = [55.84 66.27 62.38 69.87 81.37 88.17 85.26 91.93 98.7 97.84 83.22]

Price_Ret = zeros(length(P)-1)
for i in 1:length(P)-1
    Price_Ret[i] = P[i+1]/P[i]-1
end

Div = [.0298 .0337 .0317 .0301 .03 .034 .032 .0307 .0324 .0383]'
Ret = Price_Ret+Div

mean_Ret = mean(Ret)

Ret_dev = Ret.-mean_Ret
Mkt_var = (Ret.-mean_Ret).^2

var = repeat([1/10], 10)'*(Ret+Div.-mean(Ret+Div)).^2
sd = sqrt.(var)

1×1 Matrix{Float64}:
 0.09601435087790917

---------
#### 5
For the data in Table Q5 calculate the items indicated.

Another bullshit long Q that's not hard just annoying, with a huge table I cba to copy in.

----------
#### 6
What are the assumptions sufficient to guarantee that the market portfolio is an efficient
portfolio?


-----------
#### 7 
In the CAPM is there any way to identify the investors who are more risk averse? Explain.
How would your answer change if there were not a riskless asset?


--------
#### 8
Given risk-free borrowing and lending, efficient portfolios have no unsystematic risk. True or
false? Explain.


---------
#### 9
What is the beta of an efficient portfolio with $E(R_j) = 20\%$ if $R_f = 5\%$, $E(R_m) = 15\%$, and $σ_m = 20\%$? What is its $σ_j$? What is its correlation with the market?


---------- 
#### 10
Given the facts of Problem 9, and that the common stock of the Rapid Rolling Corporation has $E(R_k) = 25\%$ and $σ^2_k = 52\%$, what is the systematic risk of the common stock? What is its unsystematic risk?


------------
#### 11
(a) If the expected rate of return on the market portfolio is 14% and the risk-free rate is
6%, find the beta for a portfolio that has expected rate of return of 10%. What assumptions
concerning this portfolio and/or market conditions do you need to make to calculate the
portfolio’s beta?

(b) What percentage of this portfolio must an individual put into the market portfolio in
order to achieve an expected return of 10%?

-----------
#### 12 
You believe that the Beta Alpha Watch Company will be worth $100 per share one year from
now. How much are you willing to pay for one share today if the risk-free rate is 8%, the expected
rate of return on the market is 18%, and the company’s beta is 2.0?



----------
#### 13
Given the following variance-covariance matrix and expected returns vector (for assets X
and Y , respectively) for a two-asset world:
$$
∑ = \left[\begin{array}{cc}
.01 & 0 \\ 0 & .0064
\end{array}\right]
$$
$$
\bar{R}^′_I = \left[\begin{array}{cc}
.2 & .1
\end{array}\right]
$$


(a) What is the expected return of a zero-beta portfolio, given that 50% of the index portfolio is invested in asset X and asset Y ?

(b) What is the vector of weights in the global minimum-variance portfolio?

(c) What is the covariance between the global minimum-variance portfolio and the zero- beta portfolio?

(d) What is the equation of the market line?


----------
#### 14 
Given the following variance-covariance matrix, calculate the covariance between portfolio A, which has 10% in asset 1 and 90% in asset 2, and portfolio B, which has 60% in asset 1 and 40% in asset 2:
$$
∑ = \left[\begin{array}{cc}
.01 & -.02 \\ -.02 & .04
\end{array}\right]
$$

-------------
#### 15

Suppose that securities are priced as if they are traded in a two-parameter economy. You have
forecast the correlation coefficient between the rate of return on Knowlode Mutual Fund and the
market portfolio at .8. Your forecast of the standard deviations of the rates of return are .25 for
Knowlode, and .20 for the market portfolio. How would you combine the Knowlode Fund and a
riskless security to obtain a portfolio with a beta of 1.6?

--------
#### 16
You currently have 50% of your wealth in a risk-free asset and 50% in the four assets below:

| Asset | ER on Asset i (%) | $β_i$ | % Invested in Asset i|
| --- | --- | --- | --- |
| i = 1 | 7.6  | .2 | 10 |
| i = 2 | 12.4 | .8 | 10 |
| i = 3 | 15.6 | 1.3 | 10 |
| i = 4 | 18.8 | 1.6 | 20 |

If you want an expected rate of return of 12%, you can obtain it by selling some of your holdings
of the risk-free asset and using the proceeds to buy the equally weighted market portfolio. If this
is the way you decide to revise your portfolio, what will the set of weights in your portfolio be? If
you hold only the risk-free asset and the market portfolio, what set of weights would give you an
expected 12% return?


------------
#### 17
The market price of a security is \$40, the security’s expected rate of return is 13%, the riskless rate of interest is 7%, and the market risk premium, $\left[E(R_m) − R_f\right]$, is 8%. What will be
the security’s current price if its expected future payoff remains the same but the covariance of its
rate of return with the market portfolio doubles?


----------
#### 18
Suppose you are the manager of an investment fund in a two-parameter economy. Given the
following forecast:

$$E(R_m) = .16, σ(R_m) = .20, R_f = .08$$

(a) Would you recommend investment in a security with $E(R_j) = .12$ and $COV(R_j, R_m ) = .01$? 

(Note: Assume that this price change has no significant effect on the position of the security market line.)

(b) Suppose that in the next period security Rj has earned only 5% over the preceding period. How would you explain this ex post return?


---------
#### 19

Why is the separation principle still valid in a world with

(a) nonmarketable assets?

(b) a nonstochastic risk-free rate?


#### 20
Assume that the mean-variance opportunity set is constructed from only two risky assets, A
and B. Their variance-covariance matrix is
$$
∑ = \left[\begin{array}{cc}
.0081 & 0 \\ 0 & .0025
\end{array}\right]
$$
Asset A has an expected return of 30%, and Asset B has an expected return of 20%. Answer the
following questions:

(a) Suppose investor I chooses his “market portfolio” to consist of 75% in asset A and 25% in asset B, whereas investor J chooses a different “market portfolio” with 50% in asset A and 50% in asset B. 

Weights chosen by I are $\left[\begin{array}{cc} .75 & .25 \end{array}\right]$.

Weights chosen by J are $\left[\begin{array}{cc} .5 & .5 \end{array}\right]$. 

Given these facts, what beta will each investor calculate for asset A?

(b) Given your answer to part (a), which of the following is true and why?

1. Investor I will require a higher rate of return on asset A than will investor J.
2. They will both require the same return on asset A.
3. Investor J will require a higher rate of return on asset A than will investor I.

(c) Compute the zero-beta portfolios and the equations for the security market line for each investor.


----------
#### 21
Ms. Bethel, manager of the Humongous Mutual Fund, knows that her fund currently is well
diversified and that it has a CAPM beta of 1.0. The risk-free rate is 8% and the CAPM risk premium,

$[E(R_m) − R_f]$, is 6.2%. She has been learning about measures of risk and knows that there are (at least) two factors: changes in industrial production index, $\bar{δ}_1$, and unexpected inflation, $\bar{δ}_2$ . The APT equation is:

$$ E(R_i) − R_f = \left[\bar{δ}_1 − R_f\right] \cdot b_{i1} + \left[\bar{δ}_2 − R_f\right] \cdot b_{i2}, $$

$$ E(Ri ) = .08 + \left[.05\right] \cdot b_{i1} + \left[.11\right] \cdot b_{i2}$$

(a) If her portfolio currently has a sensitivity to the first factor of b p1 = −.5, what is its sensitivity to unexpected inflation?

(b) If she rebalances her portfolio to keep the same expected return but reduce her exposure to inflation to zero (i.e., b p2 = 0), what will its sensitivity to the first factor become?